# Python 2

In [1]:
import numpy as np

from LHCO_reader import LHCO_reader

import math
import os

# Plotting library
import matplotlib.pyplot as plt

import pandas as pd

In [2]:
!ls ../data/Signal/lhco/*.lhco

../data/Signal/lhco/BP0.lhco
../data/Signal/lhco/BP10_01.lhco
../data/Signal/lhco/BP10_02.lhco
../data/Signal/lhco/BP11_01.lhco
../data/Signal/lhco/BP11_02.lhco
../data/Signal/lhco/BP12_01.lhco
../data/Signal/lhco/BP12_02.lhco
../data/Signal/lhco/BP13_01.lhco
../data/Signal/lhco/BP13_02.lhco
../data/Signal/lhco/BP14_01.lhco
../data/Signal/lhco/BP14_02.lhco
../data/Signal/lhco/BP15_01.lhco
../data/Signal/lhco/BP15_02.lhco
../data/Signal/lhco/BP1.lhco
../data/Signal/lhco/BP1_run100k_10.lhco
../data/Signal/lhco/BP1_run100k_11.lhco
../data/Signal/lhco/BP1_run100k_12.lhco
../data/Signal/lhco/BP1_run100k_13.lhco
../data/Signal/lhco/BP1_run100k_14.lhco
../data/Signal/lhco/BP1_run100k_15.lhco
../data/Signal/lhco/BP1_run100k_16.lhco
../data/Signal/lhco/BP1_run100k_17.lhco
../data/Signal/lhco/BP1_run100k_18.lhco
../data/Signal/lhco/BP1_run100k_19.lhco
../data/Signal/lhco/BP1_run100k_1.lhco
../data/Signal/lhco/BP1_run100k_20.lhco
../data/Signal/lhco/BP1_run100k_2.lhco
../data/Signal/lhco/BP1_run1

## Signal

In [3]:
signal_NLO  = 1.25
luminosidad = 100

cross_section_bps    = np.asarray([190, 189, 189, 92, 92, 92, 49, 48, 48, 27, 27, 27, 16, 16, 16])
branching_ratios_bps = np.asarray([0.14, 0.37, 0.73, 0.15, 0.36, 0.67, 0.16, 0.36, 0.62, 0.17, 0.35, 0.58, 0.16, 0.32, 0.52])

signal_cross = cross_section_bps * branching_ratios_bps *  0.2134 # fb

In [15]:
# PARA BPs de RO. 1,2,3,4

Folder    = '../data/Signal/lhco/'
datFolder = '../data/Signal/'


signal_photon  = []
signal_jet     = []
signal_lepton  = []
signal_MET     = []
signal_hadinfo = []

signal_runs    = 21
signal_cut_ev  = 0
signal_init_ev = 0

counter_bad_jets = 0
contador_tau     = 0
for ii in range(1, signal_runs):
    print(ii) 
    inputevents = LHCO_reader.Events(f_name = Folder + "BP4_run100k_" + str(ii) + ".lhco")

    signal_init_ev += len(inputevents)
    print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')

    for i in range(len(inputevents)):

        # at least one: photon, jet and lepton (electron or muon)
        if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
            if inputevents[i]["jet"][0]["PT"] > 100 and -4.5 < inputevents[i]["jet"][0]["eta"] < 4.5:
                if inputevents[i]["photon"][0]["PT"] > 10 and -2.37 < inputevents[i]["photon"][0]["eta"] < 2.37:
                    if inputevents[i]["MET"][0]["PT"] > 100:
                        
                        # lepton info
                        if (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] > 0):
                            if inputevents[i]["electron"][0]["PT"] > inputevents[i]["muon"][0]["PT"]: 
                                flag = 'electron'
                                eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                            else: 
                                flag = 'muon'
                                eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron
                        elif (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] == 0): 
                            flag = 'electron'
                            eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                        elif (inputevents[i].number()["muon"] > 0) and (inputevents[i].number()["electron"] == 0): 
                            flag = 'muon'
                            eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron

                        if (-2.47 < eta_leading_lepton < -1.52) or (-1.37 < eta_leading_lepton < 1.37) or (1.52 < eta_leading_lepton < 2.47): 
                            
                            # how many jets (total) and total hadronic pT
                            flag_bad_jets = 0                            
                            num_jets = 0
                            HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
                            HT = 0 # total HT, defined as the scalar sum of the pT of all reconstructed particles

                            for j in range(inputevents[i].number()["jet"]):
                                if (inputevents[i]["jet"][j]["PT"] > 20) and (-4.5 < inputevents[i]["jet"][j]["eta"] < 4.5):
                                    num_jets += 1
                                    HT_had += inputevents[i]["jet"][j]["PT"]
                                    HT += inputevents[i]["jet"][j]["PT"]
                                if (np.abs(inputevents[i]["jet"][j]["eta"]) > 4.5) or (inputevents[i]["jet"][j]["PT"] < 20):
                                    flag_bad_jets = 1
                            
                            # how many photons                         
                            num_photons = 0
                            for j in range(inputevents[i].number()["photon"]):
                                if inputevents[i]["photon"][j]["PT"] > 10 and -2.37 < inputevents[i]["photon"][j]["eta"] < 2.37:
                                    num_photons = num_photons + 1
                                    HT += inputevents[i]["photon"][j]["PT"]
                                    
                            # how many muons                        
                            num_muons = 0
                            for j in range(inputevents[i].number()["muon"]):
                                if inputevents[i]["muon"][j]["PT"] > 10 and -2.7 < inputevents[i]["muon"][j]["eta"] < 2.7:
                                    num_muons = num_muons + 1
                                    HT += inputevents[i]["muon"][j]["PT"]
                                    
                            # how many electrons                        
                            num_electrons = 0
                            for j in range(inputevents[i].number()["electron"]):
                                if inputevents[i]["electron"][j]["PT"] > 10:
                                    eta = inputevents[i]["electron"][j]["eta"]
                                    if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                        num_electrons = num_electrons + 1
                                        HT += inputevents[i]["electron"][j]["PT"]
                                    
                            if flag_bad_jets == 1: counter_bad_jets += 1 
                            if flag_bad_jets == 0:
                                num_taus = 0
                                if inputevents[i].number()["tau"] > 0: 
                                    contador_tau = contador_tau + 1
                                    # how many taus                    
                                    for j in range(inputevents[i].number()["tau"]):
                                        if inputevents[i]["tau"][j]["PT"] > 20:
                                            eta = inputevents[i]["tau"][j]["eta"]
                                            if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                                num_taus = num_taus + 1
                                                HT += inputevents[i]["tau"][j]["PT"]

                                # photon info:
                                with open(datFolder + 'BP4_photon_5.dat', 'a') as f:
                                    f.write('%f %f %f %d\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"], num_photons))

                                # jet info:
                                with open(datFolder + 'BP4_jet_5.dat', 'a') as f:
                                    f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                                # lepton info
                                num_leptons = num_electrons + num_muons
                                if flag == 'electron':
                                    with open(datFolder + 'BP4_lepton_5.dat', 'a') as f:
                                        f.write('%f %f %f %d %d\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"], num_leptons, num_taus))
                                else:
                                    with open(datFolder + 'BP4_lepton_5.dat', 'a') as f:
                                        f.write('%f %f %f %d %d\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"], num_leptons, num_taus))

                                # MET (eta=0 by def, but I save phi because I have many jets)
                                with open(datFolder + 'BP4_MET_5.dat', 'a') as f:
                                    f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                                # hadronic info
                                with open(datFolder + 'BP4_hadinfo_5.dat', 'a') as f:
                                    f.write('%f %f %d\n' % (HT, HT_had, num_jets))

                                # total events that passed the cuts
                                signal_cut_ev += 1

print('Num initial events: ', signal_init_ev)
print('Total pasaron los cortes: ', signal_cut_ev)
print('Hay: ', contador_tau, ' taus')
print('Hay: ', counter_bad_jets, ' eventos con al menos un jet malo')


cross       = signal_cross[4-1] * signal_NLO
aceptancia  = signal_cut_ev / (1. * signal_init_ev)

signal_fidcross = cross * aceptancia
signal_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', signal_fidcross)
print('Events expected: ', signal_expected)

1
Esta run tuvo 100000 eventos en el lhco
2
Esta run tuvo 100000 eventos en el lhco
3
Esta run tuvo 100000 eventos en el lhco
4
Esta run tuvo 100000 eventos en el lhco
5
Esta run tuvo 100000 eventos en el lhco
6
Esta run tuvo 100000 eventos en el lhco
7
Esta run tuvo 100000 eventos en el lhco
8
Esta run tuvo 100000 eventos en el lhco
9
Esta run tuvo 100000 eventos en el lhco
10
Esta run tuvo 100000 eventos en el lhco
11
Esta run tuvo 100000 eventos en el lhco
12
Esta run tuvo 100000 eventos en el lhco
13
Esta run tuvo 100000 eventos en el lhco
14
Esta run tuvo 100000 eventos en el lhco
15
Esta run tuvo 100000 eventos en el lhco
16
Esta run tuvo 100000 eventos en el lhco
17
Esta run tuvo 100000 eventos en el lhco
18
Esta run tuvo 100000 eventos en el lhco
19
Esta run tuvo 100000 eventos en el lhco
20
Esta run tuvo 100000 eventos en el lhco
('Num initial events: ', 2000000)
('Total pasaron los cortes: ', 603424)
('Hay: ', 7971, ' taus')
('Hay: ', 5812, ' eventos con al menos un jet malo'

In [4]:
# PARA BPs de ERNEST
Folder    = '../data/Signal/lhco/'
datFolder = '../data/Signal/'


signal_photon  = []
signal_jet     = []
signal_lepton  = []
signal_MET     = []
signal_hadinfo = []

signal_bps = 15
for ii in range(5, signal_bps + 1):
    print(ii)
    counter_bad_jets = 0
    contador_tau = 0
    if ii not in [6,7,8,9,10,11,12,13,14,15]: # BPs que ya fueron analizados
        signal_cut_ev  = 0
        signal_init_ev = 0
        for jj in range(2):
            inputevents = LHCO_reader.Events(f_name = Folder + "BP" + str(ii) + "_0" + str(jj + 1) + ".lhco")

            signal_init_ev = len(inputevents) + signal_init_ev
            print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')

            for i in range(len(inputevents)):

                # at least one: photon, jet and lepton (electron or muon)
                if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
                    if inputevents[i]["jet"][0]["PT"] > 100 and -4.5 < inputevents[i]["jet"][0]["eta"] < 4.5:
                        if inputevents[i]["photon"][0]["PT"] > 10 and -2.37 < inputevents[i]["photon"][0]["eta"] < 2.37:
                            if inputevents[i]["MET"][0]["PT"] > 100:
                                # lepton info
                                if (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] > 0):
                                    if inputevents[i]["electron"][0]["PT"] > inputevents[i]["muon"][0]["PT"]: 
                                        flag = 'electron'
                                        eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                                    else: 
                                        flag = 'muon'
                                        eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron
                                elif (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] == 0): 
                                    flag = 'electron'
                                    eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                                elif (inputevents[i].number()["muon"] > 0) and (inputevents[i].number()["electron"] == 0): 
                                    flag = 'muon'
                                    eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron

                                if (-2.47 < eta_leading_lepton < -1.52) or (-1.37 < eta_leading_lepton < 1.37) or (1.52 < eta_leading_lepton < 2.47): 

                                    # how many jets (total) and total hadronic pT
                                    flag_bad_jets = 0                            
                                    num_jets = 0
                                    HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
                                    HT = 0 # total HT, defined as the scalar sum of the pT of all reconstructed particles

                                    for j in range(inputevents[i].number()["jet"]):
                                        if (inputevents[i]["jet"][j]["PT"] > 20) and (-4.5 < inputevents[i]["jet"][j]["eta"] < 4.5):
                                            num_jets += 1
                                            HT_had += inputevents[i]["jet"][j]["PT"]
                                            HT += inputevents[i]["jet"][j]["PT"]
                                        if (np.abs(inputevents[i]["jet"][j]["eta"]) > 4.5) or (inputevents[i]["jet"][j]["PT"] < 20):
                                            flag_bad_jets = 1

                                    # how many photons                         
                                    num_photons = 0
                                    for j in range(inputevents[i].number()["photon"]):
                                        if inputevents[i]["photon"][j]["PT"] > 10 and -2.37 < inputevents[i]["photon"][j]["eta"] < 2.37:
                                            num_photons = num_photons + 1
                                            HT += inputevents[i]["photon"][j]["PT"]

                                    # how many muons                        
                                    num_muons = 0
                                    for j in range(inputevents[i].number()["muon"]):
                                        if inputevents[i]["muon"][j]["PT"] > 10 and -2.7 < inputevents[i]["muon"][j]["eta"] < 2.7:
                                            num_muons = num_muons + 1
                                            HT += inputevents[i]["muon"][j]["PT"]

                                    # how many electrons                        
                                    num_electrons = 0
                                    for j in range(inputevents[i].number()["electron"]):
                                        if inputevents[i]["electron"][j]["PT"] > 10:
                                            eta = inputevents[i]["electron"][j]["eta"]
                                            if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                                num_electrons = num_electrons + 1
                                                HT += inputevents[i]["electron"][j]["PT"]

                                    if flag_bad_jets == 1: counter_bad_jets += 1 
                                    if flag_bad_jets == 0:
                                        num_taus = 0
                                        if inputevents[i].number()["tau"] > 0: 
                                            contador_tau = contador_tau + 1
                                            # how many taus                    
                                            for j in range(inputevents[i].number()["tau"]):
                                                if inputevents[i]["tau"][j]["PT"] > 20:
                                                    eta = inputevents[i]["tau"][j]["eta"]
                                                    if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                                        num_taus = num_taus + 1
                                                        HT += inputevents[i]["tau"][j]["PT"]

                                        # photon info:
                                        with open(datFolder + "BP" + str(ii) + '_photon_5.dat', 'a') as f:
                                            f.write('%f %f %f %d\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"], num_photons))

                                        # jet info:
                                        with open(datFolder + "BP" + str(ii) + '_jet_5.dat', 'a') as f:
                                            f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                                        # lepton info
                                        num_leptons = num_electrons + num_muons
                                        if flag == "electron":
                                            with open(datFolder + "BP" + str(ii) + '_lepton_5.dat', 'a') as f:
                                                f.write('%f %f %f %d %d\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"], num_leptons,num_taus))
                                        else:
                                            with open(datFolder + "BP" + str(ii) + '_lepton_5.dat', 'a') as f:
                                                f.write('%f %f %f %d %d\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"], num_leptons,num_taus))

                                        # MET (eta=0 by def, but I save phi because I have many jets)
                                        with open(datFolder + "BP" + str(ii) + '_MET_5.dat', 'a') as f:
                                            f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                                        # hadronic info
                                        with open(datFolder + "BP" + str(ii) + '_hadinfo_5.dat', 'a') as f:
                                            f.write('%f %f %d\n' % (HT, HT_had, num_jets))

                                        # total events that passed the cuts
                                        signal_cut_ev += 1
        print('End of BP' + str(ii) + ' analysis')
        inputevents = 0 # Just to clean the memory

        print('Num initial events: ', signal_init_ev)
        print('Total pasaron los cortes: ', signal_cut_ev)
        print('Hay: ', contador_tau, ' taus')
        print('Hay: ', counter_bad_jets, ' eventos con al menos un jet malo')
        cross       = signal_cross[ii-1] * signal_NLO
        aceptancia  = signal_cut_ev / (1. * signal_init_ev)

        signal_fidcross = cross * aceptancia
        signal_expected = cross * aceptancia * luminosidad

        print('fiducial cross section: ', signal_fidcross)
        print('Events expected: ', signal_expected)

5


/home/martinrios/.virtualenvs/pheno2.7/lib/python2.7/site-packages/LHCO_reader/LHCO_reader.py:359: UserWarning: Couldn't read total number of events from LHCO
  warnings.warn("Couldn't read total number of events from LHCO")


Esta run tuvo 1000000 eventos en el lhco
Esta run tuvo 1000000 eventos en el lhco
End of BP5 analysis
('Num initial events: ', 2000000)
('Total pasaron los cortes: ', 533112)
('Hay: ', 7048, ' taus')
('Hay: ', 5187, ' eventos con al menos un jet malo')
('fiducial cross section: ', 2.35495828656)
('Events expected: ', 235.49582865600001)
6
7
8
9
10
11
12
13
14
15


In [20]:
signal_cross = 48 * 0.36 *  0.2134 # fb
signal_init_sim_eve = 1000000
signal_cut_ev       = 293092
signal_NLO          = 1.25

cross       = signal_cross * signal_NLO
aceptancia  = signal_cut_ev / (1. * signal_init_sim_eve)
luminosidad = 100

signal_fidcross = cross * aceptancia
signal_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', signal_fidcross)
print('Events expected: ', signal_expected)

('fiducial cross section: ', 1.35098998848)
('Events expected: ', 135.098998848)


## Backgrounds

### pp > tt $\gamma$

In [21]:
Folder    = '/home/martinrios/Downloads/MG5aMC_LTS_2.9.9/MG5_aMC_v2_9_9/pptottbargamma/'
datFolder = '../data/ttGamma/'

ttgamma_cross = 0.144 #[pb] # Integrated cross section without matching
ttgamma_runs  = 150
ttgamma_NLO   = 1.46 # k-factor [14 TeV] CHECK THIS
ttgamma_init_sim_eve = 100000 * ttgamma_runs

ttgamma_init_ev = 0
ttgamma_cut_ev  = 0

ttgamma_photon  = []
ttgamma_jet     = []
ttgamma_lepton  = []
ttgamma_MET     = []
ttgamma_hadinfo = []


counter_bad_jets = 0
contador_tau = 0
for ii in range(1, 1 + ttgamma_runs):
    print(ii)
    if ii not in [0]:
        inputevents = LHCO_reader.Events(f_name = Folder + "run100k_" + str(ii) + ".lhco")

        ttgamma_init_ev += len(inputevents)
        print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')
        
        for i in range(len(inputevents)):

            num_jets = 0
    #        num_btag = 0
            HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets


            # at least one: photon, jet and lepton (electron or muon)
            if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
                if inputevents[i]["jet"][0]["PT"] > 100 and -4.5 < inputevents[i]["jet"][0]["eta"] < 4.5:
                    if inputevents[i]["photon"][0]["PT"] > 10 and -2.37 < inputevents[i]["photon"][0]["eta"] < 2.37:
                        if inputevents[i]["MET"][0]["PT"] > 100:
                            # lepton info
                            if (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] > 0):
                                if inputevents[i]["electron"][0]["PT"] > inputevents[i]["muon"][0]["PT"]: 
                                    flag = 'electron'
                                    eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                                else: 
                                    flag = 'muon'
                                    eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron
                            elif (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] == 0): 
                                flag = 'electron'
                                eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                            elif (inputevents[i].number()["muon"] > 0) and (inputevents[i].number()["electron"] == 0): 
                                flag = 'muon'
                                eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron

                            if (-2.47 < eta_leading_lepton < -1.52) or (-1.37 < eta_leading_lepton < 1.37) or (1.52 < eta_leading_lepton < 2.47): 

                                # how many jets (total) and total hadronic pT
                                flag_bad_jets = 0                            
                                num_jets = 0
                                HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
                                HT = 0 # total HT, defined as the scalar sum of the pT of all reconstructed particles

                                for j in range(inputevents[i].number()["jet"]):
                                    if (inputevents[i]["jet"][j]["PT"] > 20) and (-4.5 < inputevents[i]["jet"][j]["eta"] < 4.5):
                                        num_jets += 1
                                        HT_had += inputevents[i]["jet"][j]["PT"]
                                        HT += inputevents[i]["jet"][j]["PT"]
                                    if (np.abs(inputevents[i]["jet"][j]["eta"]) > 4.5) or (inputevents[i]["jet"][j]["PT"] < 20):
                                        flag_bad_jets = 1

                                # how many photons                         
                                num_photons = 0
                                for j in range(inputevents[i].number()["photon"]):
                                    if inputevents[i]["photon"][j]["PT"] > 10 and -2.37 < inputevents[i]["photon"][j]["eta"] < 2.37:
                                        num_photons = num_photons + 1
                                        HT += inputevents[i]["photon"][j]["PT"]

                                # how many muons                        
                                num_muons = 0
                                for j in range(inputevents[i].number()["muon"]):
                                    if inputevents[i]["muon"][j]["PT"] > 10 and -2.7 < inputevents[i]["muon"][j]["eta"] < 2.7:
                                        num_muons = num_muons + 1
                                        HT += inputevents[i]["muon"][j]["PT"]

                                # how many electrons                        
                                num_electrons = 0
                                for j in range(inputevents[i].number()["electron"]):
                                    if inputevents[i]["electron"][j]["PT"] > 10:
                                        eta = inputevents[i]["electron"][j]["eta"]
                                        if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                            num_electrons = num_electrons + 1
                                            HT += inputevents[i]["electron"][j]["PT"]

                                if flag_bad_jets == 1: counter_bad_jets += 1 
                                if flag_bad_jets == 0:
                                    num_taus = 0
                                    if inputevents[i].number()["tau"] > 0: 
                                        contador_tau = contador_tau + 1
                                        # how many taus                    
                                        for j in range(inputevents[i].number()["tau"]):
                                            if inputevents[i]["tau"][j]["PT"] > 20:
                                                eta = inputevents[i]["tau"][j]["eta"]
                                                if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                                    num_taus = num_taus + 1
                                                    HT += inputevents[i]["tau"][j]["PT"]

                                    # photon info:
                                    with open(datFolder + 'photon_4.dat', 'a') as f:
                                        f.write('%f %f %f %d\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"], num_photons))

                                    # jet info:
                                    with open(datFolder + 'jet_4.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                                    # lepton info
                                    num_leptons = num_electrons + num_muons
                                    if flag == "electron":
                                        with open(datFolder + 'lepton_4.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"], num_leptons, num_taus))
                                    else:
                                        with open(datFolder + 'lepton_4.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"], num_leptons, num_taus))

                                    # MET (eta=0 by def, but I save phi because I have many jets)
                                    with open(datFolder + 'MET_4.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                                    # hadronic info
                                    with open(datFolder + 'hadinfo_4.dat', 'a') as f:
                                        f.write('%f %f %d\n' % (HT, HT_had, num_jets))

                                    # total events that passed the cuts
                                    ttgamma_cut_ev += 1

print('Num initial events: ', ttgamma_init_ev)
print('Total pasaron los cortes: ', ttgamma_cut_ev)
print('Hay: ', contador_tau, ' taus')
print('Hay: ', counter_bad_jets, ' eventos con al menos un jet malo')

1
Esta run tuvo 55020 eventos en el lhco
2
Esta run tuvo 54714 eventos en el lhco
3
Esta run tuvo 55072 eventos en el lhco
4
Esta run tuvo 54976 eventos en el lhco
5
Esta run tuvo 54935 eventos en el lhco
6
Esta run tuvo 54733 eventos en el lhco
7
Esta run tuvo 55075 eventos en el lhco
8
Esta run tuvo 55108 eventos en el lhco
9
Esta run tuvo 55151 eventos en el lhco
10
Esta run tuvo 55887 eventos en el lhco
11
Esta run tuvo 54990 eventos en el lhco
12
Esta run tuvo 54880 eventos en el lhco
13
Esta run tuvo 55077 eventos en el lhco
14
Esta run tuvo 55014 eventos en el lhco
15
Esta run tuvo 55043 eventos en el lhco
16
Esta run tuvo 55097 eventos en el lhco
17
Esta run tuvo 54874 eventos en el lhco
18
Esta run tuvo 55182 eventos en el lhco
19
Esta run tuvo 55031 eventos en el lhco
20
Esta run tuvo 54877 eventos en el lhco
21
Esta run tuvo 54937 eventos en el lhco
22
Esta run tuvo 54992 eventos en el lhco
23
Esta run tuvo 55212 eventos en el lhco
24
Esta run tuvo 43581 eventos en el lhco
2

In [13]:
ttgamma_cut_ev = 2052938
ttgamma_init_ev = 8184997

In [22]:
ttgamma_cross = 0.144 #[pb]# Integrated cross section without matching
ttgamma_runs  = 150
ttgamma_NLO   = 1.46 # k-factor [14 TeV] 
ttgamma_init_sim_eve = 100000 * ttgamma_runs

In [23]:
cross       = ttgamma_cross * 1000 * ttgamma_NLO
aceptancia  = ttgamma_cut_ev / (1. * ttgamma_init_sim_eve)
luminosidad = 100

ttgamma_fidcross = cross * aceptancia
ttgamma_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', ttgamma_fidcross)
print('Events expected: ', ttgamma_expected)

('fiducial cross section: ', 28.773979008)
('Events expected: ', 2877.3979008)


### p p > W $\gamma$ > $\nu$ + l + $\gamma$

In [24]:
Folder    = '/home/martinrios/Downloads/MG5aMC_LTS_2.9.9/MG5_aMC_v2_9_9/pptoWgamma/'
datFolder = '../data/Wgamma/'

Wgamma_cross = 0.944 #[pb]  # Integrated cross section without matching
Wgamma_runs  = 137
Wgamma_NLO   = 2.37 # k-factor [14 TeV]
Wgamma_init_sim_eve = 100000 * Wgamma_runs

Wgamma_init_ev = 0
Wgamma_cut_ev  = 0

Wgamma_photon  = []
Wgamma_jet     = []
Wgamma_lepton  = []
Wgamma_MET     = []
Wgamma_hadinfo = []


counter_bad_jets = 0
contador_tau = 0
for ii in range(1, 1 + Wgamma_runs):
    print(ii)
    if ii not in [0]:
        inputevents = LHCO_reader.Events(f_name = Folder + "run100k_" + str(ii) + ".lhco")

        Wgamma_init_ev += len(inputevents)
        print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')
        
        for i in range(len(inputevents)):

            num_jets = 0
    #        num_btag = 0
            HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets


            # at least one: photon, jet and lepton (electron or muon)
            if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
                if inputevents[i]["jet"][0]["PT"] > 100 and -4.5 < inputevents[i]["jet"][0]["eta"] < 4.5:
                    if inputevents[i]["photon"][0]["PT"] > 10 and -2.37 < inputevents[i]["photon"][0]["eta"] < 2.37:
                        if inputevents[i]["MET"][0]["PT"] > 100:
                            # lepton info
                            if (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] > 0):
                                if inputevents[i]["electron"][0]["PT"] > inputevents[i]["muon"][0]["PT"]: 
                                    flag = 'electron'
                                    eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                                else: 
                                    flag = 'muon'
                                    eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron
                            elif (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] == 0): 
                                flag = 'electron'
                                eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                            elif (inputevents[i].number()["muon"] > 0) and (inputevents[i].number()["electron"] == 0): 
                                flag = 'muon'
                                eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron

                            if (-2.47 < eta_leading_lepton < -1.52) or (-1.37 < eta_leading_lepton < 1.37) or (1.52 < eta_leading_lepton < 2.47): 

                                # how many jets (total) and total hadronic pT
                                flag_bad_jets = 0                            
                                num_jets = 0
                                HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
                                HT = 0 # total HT, defined as the scalar sum of the pT of all reconstructed particles

                                for j in range(inputevents[i].number()["jet"]):
                                    if (inputevents[i]["jet"][j]["PT"] > 20) and (-4.5 < inputevents[i]["jet"][j]["eta"] < 4.5):
                                        num_jets += 1
                                        HT_had += inputevents[i]["jet"][j]["PT"]
                                        HT += inputevents[i]["jet"][j]["PT"]
                                    if (np.abs(inputevents[i]["jet"][j]["eta"]) > 4.5) or (inputevents[i]["jet"][j]["PT"] < 20):
                                        flag_bad_jets = 1

                                # how many photons                         
                                num_photons = 0
                                for j in range(inputevents[i].number()["photon"]):
                                    if inputevents[i]["photon"][j]["PT"] > 10 and -2.37 < inputevents[i]["photon"][j]["eta"] < 2.37:
                                        num_photons = num_photons + 1
                                        HT += inputevents[i]["photon"][j]["PT"]

                                # how many muons                        
                                num_muons = 0
                                for j in range(inputevents[i].number()["muon"]):
                                    if inputevents[i]["muon"][j]["PT"] > 10 and -2.7 < inputevents[i]["muon"][j]["eta"] < 2.7:
                                        num_muons = num_muons + 1
                                        HT += inputevents[i]["muon"][j]["PT"]

                                # how many electrons                        
                                num_electrons = 0
                                for j in range(inputevents[i].number()["electron"]):
                                    if inputevents[i]["electron"][j]["PT"] > 10:
                                        eta = inputevents[i]["electron"][j]["eta"]
                                        if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                            num_electrons = num_electrons + 1
                                            HT += inputevents[i]["electron"][j]["PT"]

                                if flag_bad_jets == 1: counter_bad_jets += 1 
                                if flag_bad_jets == 0:
                                    num_taus = 0
                                    if inputevents[i].number()["tau"] > 0: 
                                        contador_tau = contador_tau + 1
                                        # how many taus                    
                                        for j in range(inputevents[i].number()["tau"]):
                                            if inputevents[i]["tau"][j]["PT"] > 20:
                                                eta = inputevents[i]["tau"][j]["eta"]
                                                if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                                    num_taus = num_taus + 1
                                                    HT += inputevents[i]["tau"][j]["PT"]

                                    # photon info:
                                    with open(datFolder + 'photon_4.dat', 'a') as f:
                                        f.write('%f %f %f %d\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"], num_photons))

                                    # jet info:
                                    with open(datFolder + 'jet_4.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                                    # lepton info
                                    num_leptons = num_electrons + num_muons
                                    if flag == "electron":
                                        with open(datFolder + 'lepton_4.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"], num_leptons, num_taus))
                                    else:
                                        with open(datFolder + 'lepton_4.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"], num_leptons, num_taus))

                                    # MET (eta=0 by def, but I save phi because I have many jets)
                                    with open(datFolder + 'MET_4.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                                    # hadronic info
                                    with open(datFolder + 'hadinfo_4.dat', 'a') as f:
                                        f.write('%f %f %d\n' % (HT, HT_had, num_jets))

                                    # total events that passed the cuts
                                    Wgamma_cut_ev += 1

print('Num initial events: ', Wgamma_init_ev)
print('Total pasaron los cortes: ', Wgamma_cut_ev)
print('Hay: ', contador_tau, ' taus')
print('Hay: ', counter_bad_jets, ' eventos con al menos un jet malo')

1
Esta run tuvo 96394 eventos en el lhco
2
Esta run tuvo 96333 eventos en el lhco
3
Esta run tuvo 96331 eventos en el lhco
4
Esta run tuvo 96491 eventos en el lhco
5
Esta run tuvo 96378 eventos en el lhco
6
Esta run tuvo 96527 eventos en el lhco
7
Esta run tuvo 96453 eventos en el lhco
8
Esta run tuvo 96428 eventos en el lhco
9
Esta run tuvo 96467 eventos en el lhco
10
Esta run tuvo 96433 eventos en el lhco
11
Esta run tuvo 96472 eventos en el lhco
12
Esta run tuvo 96347 eventos en el lhco
13
Esta run tuvo 96530 eventos en el lhco
14
Esta run tuvo 96510 eventos en el lhco
15
Esta run tuvo 96353 eventos en el lhco
16
Esta run tuvo 96462 eventos en el lhco
17
Esta run tuvo 96376 eventos en el lhco
18
Esta run tuvo 96341 eventos en el lhco
19
Esta run tuvo 96392 eventos en el lhco
20
Esta run tuvo 96418 eventos en el lhco
21
Esta run tuvo 96464 eventos en el lhco
22
Esta run tuvo 96371 eventos en el lhco
23
Esta run tuvo 96395 eventos en el lhco
24
Esta run tuvo 96402 eventos en el lhco
2

In [17]:
Wgamma_cut_ev = 3579946
Wgamma_init_ev = 13210085

In [25]:
Wgamma_cross = 0.944 #[pb]  # Integrated cross section without matching
Wgamma_runs  = 137
Wgamma_NLO   = 2.37 # k-factor [14 TeV]
Wgamma_init_sim_eve = 100000 * Wgamma_runs

In [26]:
#Wgamma_cut_ev = 3810394
cross       = Wgamma_cross * 1000 * Wgamma_NLO
aceptancia  = Wgamma_cut_ev / (1. * Wgamma_init_sim_eve)
luminosidad = 100

Wgamma_fidcross = cross * aceptancia
Wgamma_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', Wgamma_fidcross)
print('Events expected: ', Wgamma_expected)

('fiducial cross section: ', 584.6234734948906)
('Events expected: ', 58462.347349489064)


In [27]:
Wgamma_init_sim_eve

13700000

### p p > W jets > $\nu$ l jets

In [21]:
# PAPER MADGRAPH for the NLO corrections
# https://arxiv.org/pdf/1405.0301.pdf

In [2]:
Folder    = '/home/martinrios/Downloads/MG5aMC_LTS_2.9.9/MG5_aMC_v2_9_9/pptowjtolnuj/'
datFolder = '../data/Wjets/'

Wjets_runs  = 500
Wjets_runs_1M = 40

Wjets_init_ev = 0
Wjets_cut_ev  = 0

Wjets_photon  = []
Wjets_jet     = []
Wjets_lepton  = []
Wjets_MET     = []
Wjets_hadinfo = []


counter_bad_jets = 0
contador_tau = 0
i1M_contador = 0

analyzed_100k_runs = 0
analyzed_1M_runs = 0
for ii in range(1, Wjets_runs + Wjets_runs_1M):
    if ii not in [19, 33, 106, 325]: # Runs que fueron cancelados
        if ii < Wjets_runs:
            f_name = Folder + "run100k_" + str(ii) + ".lhco"
            analyzed_100k_runs += 1
        else:
            i1M_contador += 1
            f_name = Folder + "run1M_" + str(i1M_contador) + ".lhco"
            analyzed_1M_runs += 1
            
        print('Reading ', f_name)
        inputevents = LHCO_reader.Events(f_name = f_name)

        Wjets_init_ev += len(inputevents)
        print('Esta run tuvo ' + str(len(inputevents)) + ' eventos en el lhco')
        
        for i in range(len(inputevents)):

            num_jets = 0
    #        num_btag = 0
            HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets


            # at least one: photon, jet and lepton (electron or muon)
            if inputevents[i].number()["photon"] > 0 and inputevents[i].number()["jet"] > 0 and (inputevents[i].number()["electron"] > 0 or inputevents[i].number()["muon"] > 0):
                if inputevents[i]["jet"][0]["PT"] > 100 and -4.5 < inputevents[i]["jet"][0]["eta"] < 4.5:
                    if inputevents[i]["photon"][0]["PT"] > 10 and -2.37 < inputevents[i]["photon"][0]["eta"] < 2.37:
                        if inputevents[i]["MET"][0]["PT"] > 100:
                            # lepton info
                            if (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] > 0):
                                if inputevents[i]["electron"][0]["PT"] > inputevents[i]["muon"][0]["PT"]: 
                                    flag = 'electron'
                                    eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                                else: 
                                    flag = 'muon'
                                    eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron
                            elif (inputevents[i].number()["electron"] > 0) and (inputevents[i].number()["muon"] == 0): 
                                flag = 'electron'
                                eta_leading_lepton = inputevents[i]["electron"][0]["eta"]
                            elif (inputevents[i].number()["muon"] > 0) and (inputevents[i].number()["electron"] == 0): 
                                flag = 'muon'
                                eta_leading_lepton = -2. # Esto es para q el corte sea solo cuando el leading lepton es un electron

                            if (-2.47 < eta_leading_lepton < -1.52) or (-1.37 < eta_leading_lepton < 1.37) or (1.52 < eta_leading_lepton < 2.47): 

                                # how many jets (total) and total hadronic pT
                                flag_bad_jets = 0                            
                                num_jets = 0
                                HT_had = 0 # hadronic activity HT, defined as the scalar sum of the pT of all reconstructed jets
                                HT = 0 # total HT, defined as the scalar sum of the pT of all reconstructed particles

                                for j in range(inputevents[i].number()["jet"]):
                                    if (inputevents[i]["jet"][j]["PT"] > 20) and (-4.5 < inputevents[i]["jet"][j]["eta"] < 4.5):
                                        num_jets += 1
                                        HT_had += inputevents[i]["jet"][j]["PT"]
                                        HT += inputevents[i]["jet"][j]["PT"]
                                    if (np.abs(inputevents[i]["jet"][j]["eta"]) > 4.5) or (inputevents[i]["jet"][j]["PT"] < 20):
                                        flag_bad_jets = 1

                                # how many photons                         
                                num_photons = 0
                                for j in range(inputevents[i].number()["photon"]):
                                    if inputevents[i]["photon"][j]["PT"] > 10 and -2.37 < inputevents[i]["photon"][j]["eta"] < 2.37:
                                        num_photons = num_photons + 1
                                        HT += inputevents[i]["photon"][j]["PT"]

                                # how many muons                        
                                num_muons = 0
                                for j in range(inputevents[i].number()["muon"]):
                                    if inputevents[i]["muon"][j]["PT"] > 10 and -2.7 < inputevents[i]["muon"][j]["eta"] < 2.7:
                                        num_muons = num_muons + 1
                                        HT += inputevents[i]["muon"][j]["PT"]

                                # how many electrons                        
                                num_electrons = 0
                                for j in range(inputevents[i].number()["electron"]):
                                    if inputevents[i]["electron"][j]["PT"] > 10:
                                        eta = inputevents[i]["electron"][j]["eta"]
                                        if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                            num_electrons = num_electrons + 1
                                            HT += inputevents[i]["electron"][j]["PT"]

                                if flag_bad_jets == 1: counter_bad_jets += 1 
                                if flag_bad_jets == 0:
                                    num_taus = 0
                                    if inputevents[i].number()["tau"] > 0: 
                                        contador_tau = contador_tau + 1
                                        # how many taus                    
                                        for j in range(inputevents[i].number()["tau"]):
                                            if inputevents[i]["tau"][j]["PT"] > 20:
                                                eta = inputevents[i]["tau"][j]["eta"]
                                                if (-2.47 < eta < -1.52) or (-1.37 < eta < 1.37) or (1.52 < eta < 2.47): 
                                                    num_taus = num_taus + 1
                                                    HT += inputevents[i]["tau"][j]["PT"]
                                    
                                    # photon info:
                                    with open(datFolder + 'photon_4.dat', 'a') as f:
                                        f.write('%f %f %f %d\n' % (inputevents[i]["photon"][0]["PT"],inputevents[i]["photon"][0]["eta"],inputevents[i]["photon"][0]["phi"], num_photons))

                                    # jet info:
                                    with open(datFolder + 'jet_4.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["jet"][0]["PT"],inputevents[i]["jet"][0]["eta"],inputevents[i]["jet"][0]["phi"]))

                                    # lepton info
                                    num_leptons = num_electrons + num_muons
                                    if flag == "electron":
                                        with open(datFolder + 'lepton_4.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["electron"][0]["PT"],inputevents[i]["electron"][0]["eta"],inputevents[i]["electron"][0]["phi"], num_leptons, num_taus))
                                    else:
                                        with open(datFolder + 'lepton_4.dat', 'a') as f:
                                            f.write('%f %f %f %d %d\n' % (inputevents[i]["muon"][0]["PT"],inputevents[i]["muon"][0]["eta"],inputevents[i]["muon"][0]["phi"], num_leptons, num_taus))

                                    # MET (eta=0 by def, but I save phi because I have many jets)
                                    with open(datFolder + 'MET_4.dat', 'a') as f:
                                        f.write('%f %f %f\n' % (inputevents[i]["MET"][0]["PT"],inputevents[i]["MET"][0]["eta"],inputevents[i]["MET"][0]["phi"]))

                                    # hadronic info
                                    with open(datFolder + 'hadinfo_4.dat', 'a') as f:
                                        f.write('%f %f %d\n' % (HT, HT_had, num_jets))

                                    # total events that passed the cuts
                                    Wjets_cut_ev += 1

print('Num initial events: ', Wjets_init_ev)
print('Total pasaron los cortes: ', Wjets_cut_ev)
print('Hay: ', contador_tau, ' taus')
print('Hay: ', counter_bad_jets, ' eventos con al menos un jet malo')

('Reading ', '/home/martinrios/Downloads/MG5aMC_LTS_2.9.9/MG5_aMC_v2_9_9/pptowjtolnuj/run100k_1.lhco')


/home/martinrios/.virtualenvs/pheno2.7/lib/python2.7/site-packages/LHCO_reader/LHCO_reader.py:359: UserWarning: Couldn't read total number of events from LHCO
  warnings.warn("Couldn't read total number of events from LHCO")


Esta run tuvo 54919 eventos en el lhco
('Reading ', '/home/martinrios/Downloads/MG5aMC_LTS_2.9.9/MG5_aMC_v2_9_9/pptowjtolnuj/run100k_2.lhco')
Esta run tuvo 54896 eventos en el lhco
('Reading ', '/home/martinrios/Downloads/MG5aMC_LTS_2.9.9/MG5_aMC_v2_9_9/pptowjtolnuj/run100k_3.lhco')
Esta run tuvo 54637 eventos en el lhco
('Reading ', '/home/martinrios/Downloads/MG5aMC_LTS_2.9.9/MG5_aMC_v2_9_9/pptowjtolnuj/run100k_4.lhco')
Esta run tuvo 54863 eventos en el lhco
('Reading ', '/home/martinrios/Downloads/MG5aMC_LTS_2.9.9/MG5_aMC_v2_9_9/pptowjtolnuj/run100k_5.lhco')
Esta run tuvo 54743 eventos en el lhco
('Reading ', '/home/martinrios/Downloads/MG5aMC_LTS_2.9.9/MG5_aMC_v2_9_9/pptowjtolnuj/run100k_6.lhco')
Esta run tuvo 54751 eventos en el lhco
('Reading ', '/home/martinrios/Downloads/MG5aMC_LTS_2.9.9/MG5_aMC_v2_9_9/pptowjtolnuj/run100k_7.lhco')
Esta run tuvo 54698 eventos en el lhco
('Reading ', '/home/martinrios/Downloads/MG5aMC_LTS_2.9.9/MG5_aMC_v2_9_9/pptowjtolnuj/run100k_8.lhco')
Esta r

In [3]:
Wjets_cross = 168.12 #[pb]  # Integrated cross section without matching
# Hay 4 corridas de 100k canceladas (19,33,106,325) y 3 corridas q fueron renombradas como 100k pero tienen 1M (81,82,83)
Wjets_runs  = analyzed_100k_runs - 3
Wjets_NLO   = 1.27 # k-factor [14 TeV]
Wjets_init_sim_eve = (100000 * Wjets_runs) + 3000000 + (1000000 * analyzed_1M_runs)

In [4]:
analyzed_100k_runs

495

In [5]:
analyzed_1M_runs

40

In [6]:
# full
print(Wjets_init_ev)
print(Wjets_cut_ev)

50384374
259345


In [21]:
Wjets_init_ev = 50384374 #40877363 #19022395 # Events after matching
Wjets_cut_ev = 259345 #210604 #98061

In [7]:
# full

cross       = Wjets_cross * 1000 * Wjets_NLO
aceptancia  = Wjets_cut_ev / (1. * Wjets_init_sim_eve)
luminosidad = 100

Wjets_fidcross = cross * aceptancia
Wjets_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', Wjets_fidcross)
print('Events expected: ', Wjets_expected)

('fiducial cross section: ', 600.5788869631236)
('Events expected: ', 60057.888696312366)


In [59]:
# Primeros 325
print(Wjets_init_ev)
print(Wjets_cut_ev)

19022395
98061


In [38]:
# 326-500
print(Wjets_init_ev)
print(Wjets_cut_ev)

9507011
48741


In [65]:
# 40 1M
print(Wjets_init_ev)
print(Wjets_cut_ev)

21309179
109626


In [60]:
# Primeros 325

cross       = Wjets_cross * 1000 * Wjets_NLO
aceptancia  = Wjets_cut_ev / (1. * Wjets_init_sim_eve)
luminosidad = 100

Wjets_fidcross = cross * aceptancia
Wjets_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', Wjets_fidcross)
print('Events expected: ', Wjets_expected)

('fiducial cross section: ', 601.6448119655172)
('Events expected: ', 60164.48119655172)


In [54]:
# 325-500

cross       = Wjets_cross * 1000 * Wjets_NLO
aceptancia  = Wjets_cut_ev / (1. * Wjets_init_sim_eve)
luminosidad = 100

Wjets_fidcross = cross * aceptancia
Wjets_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', Wjets_fidcross)
print('Events expected: ', Wjets_expected)

('fiducial cross section: ', 598.0924073793103)
('Events expected: ', 59809.24073793103)


In [66]:
# 40 1M

cross       = Wjets_cross * 1000 * Wjets_NLO
aceptancia  = Wjets_cut_ev / (1. * Wjets_init_sim_eve)
luminosidad = 100

Wjets_fidcross = cross * aceptancia
Wjets_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', Wjets_fidcross)
print('Events expected: ', Wjets_expected)

('fiducial cross section: ', 600.1669323692307)
('Events expected: ', 60016.69323692307)


In [31]:
cross       = Wjets_cross * 1000 * Wjets_NLO
aceptancia  = Wjets_cut_ev / (1. * Wjets_init_sim_eve)
luminosidad = 100

Wjets_fidcross = cross * aceptancia
Wjets_expected = cross * aceptancia * luminosidad

print('fiducial cross section: ', Wjets_fidcross)
print('Events expected: ', Wjets_expected)

('fiducial cross section: ', 592.8308799304813)
('Events expected: ', 59283.087993048124)
